In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
train = pd.read_csv("/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/train.csv")
test = pd.read_csv("/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train_Y = train['Cost'].abs()
train_X = train.drop(['Cost'], axis=1)
test_X = test
train_X

**Using Pandas Profiling for EDA**

In [ ]:
import pandas_profiling 

In [ ]:
train.profile_report()

In [ ]:
train_X = train_X.drop(['Customer Id', 'Artist Name', 'Scheduled Date', 'Delivery Date', 'Customer Location', 'Transport'], axis = 1)
test_X = test.drop(['Customer Id', 'Artist Name', 'Scheduled Date', 'Delivery Date', 'Customer Location', 'Transport'], axis = 1)

# Dealing with categorical columns having missing values

In [ ]:
# 1. Function to replace NAN values with mode value
def impute_nan_most_frequent_category(DataFrame,ColName):
    # .mode()[0] - gives first category name
     most_frequent_category=DataFrame[ColName].mode()[0]
    
    # replace nan values with most occured category
     DataFrame[ColName + "_Imputed"] = DataFrame[ColName]
     DataFrame[ColName + "_Imputed"].fillna(most_frequent_category,inplace=True)
#2. Call function to impute most occured category
for Columns in ['Material', 'Remote Location']:
    impute_nan_most_frequent_category(train_X,Columns)
    impute_nan_most_frequent_category(test_X,Columns)
    
# Display imputed result
train_X[['Material','Material_Imputed','Remote Location','Remote Location_Imputed']].head(10)
#3. Drop actual columns
train_X = train_X.drop(['Material', 'Remote Location'], axis = 1)
test_X = test_X.drop(['Material', 'Remote Location'], axis = 1)

In [ ]:
#test_X.profile_report()
train_X

In [ ]:
# Get list of categorical variables
s = (train_X.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

#### Label Encoding Categorical Data*Label Encoding Categorical Data*

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
label_X_train = train_X.copy()
label_X_test = test_X.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(train_X[col])
    label_X_test[col] = label_encoder.transform(test_X[col])

#### *Imputing missing values*

In [ ]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(label_X_train))
imputed_X_test = pd.DataFrame(my_imputer.transform(label_X_test))

# Imputation removed column names; put them back
imputed_X_train.columns = train_X.columns
imputed_X_test.columns = test_X.columns

In [ ]:
imputed_X_train

#### Data Normalisation

In [ ]:
import sklearn
scaler = sklearn.preprocessing.StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(imputed_X_train), columns=imputed_X_train.columns)
X_test = pd.DataFrame(scaler.transform(imputed_X_test), columns=imputed_X_test.columns)

#### RandomForest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, train_Y)

In [ ]:
y_prediction_rf = rf.predict(X_test)
y_prediction_rf

#### Tuning RF

In [ ]:
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_grid = [
{'n_estimators': [50,100,250,500], 
 'max_depth': [10, 50, 100], 'bootstrap': [True, False]}
]

In [ ]:
random_search_forest = RandomizedSearchCV(rf, param_grid, cv=10)
random_search_forest.fit(X_train, train_Y)

In [ ]:
tuned_rf_best_random = random_search_forest.best_estimator_
tuned_rf_best_random

In [ ]:
tuned_rf_best_random.fit(X_train, train_Y)

In [ ]:
tuned_rf_random_pred = tuned_rf_best_random.predict(X_test)
tuned_rf_random_pred

In [ ]:
df = pd.DataFrame()

In [ ]:
df['Customer Id'] = test['Customer Id']
df['Cost'] = y_prediction_rf
df['Cost'] = df['Cost'].abs()
df

In [ ]:
df.to_csv('prediction1.csv', index = False)